In [6]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import cv2
import random
batch_size = 3

In [8]:
def add_noise(image, mean=0, val=0.01):
    size = image.shape
    image = image / 255
    gauss = np.random.normal(mean, val ** 0.5, size)
    noise = image + gauss
    return gauss,noise

In [9]:
def all(old_train_path,new_train_path):
    fold_old = os.listdir(old_train_path) #假设old_path都在train目录
    #newpath的定位也定在train下
    for fold in fold_old:
        fold_new_path = new_train_path + '/' + fold
        if not os.path.exists(fold_new_path):
            os.makedirs(fold_new_path)
        old_path = old_train_path + '/' + fold
        #图片的一个list
        i = 0
        j = 100
        img_old = os.listdir(old_path)
        for img_old in tqdm(img_old):
            k = random.uniform(0,1)
            if k>0 and k<=0.2:
            #增加噪声
                img = cv2.imread(old_path + '/' + img_old)
                color_gauss,color_noisy_image = add_noise(img)
                save_path = (fold_new_path + '/' +'{0}'+'.jpg').format(j)   
                j += 1
                color_noisy_image = color_noisy_image*255
                cv2.imencode(".jpg", color_noisy_image)[1].tofile(save_path)
            
            #伽马变换
            elif k>0.20 and k<=0.4:
                img = cv2.imread(old_path + '/' + img_old)
                normImg = lambda x: 255. * (x-x.min()) / (x.max()-x.min()+1e-6)  # 归一化为 [0,255]
                gammaList = random.uniform(0.7,1.5)
                imgGamma = np.power(img, gammaList)
                imgGamma = np.uint8(normImg(imgGamma))
                save_path = (fold_new_path + '/' +'{0}'+'.jpg').format(j)
                j+=1
                cv2.imencode(".jpg", imgGamma)[1].tofile(save_path)

            #旋转变换
            elif k>0.4 and k <= 0.8:
                data_gen = ImageDataGenerator(rotation_range=3,
                                    width_shift_range=0.02,
                                    height_shift_range=0.02,
                                    horizontal_flip=True,
                                    zoom_range=0.03,
                                    fill_mode='nearest',
                                    rescale=1. / 255,
                                    data_format='channels_last')
                img=load_img(old_path+'/'+img_old)
                x = img_to_array(img,data_format="channels_last")   #图片转化成array类型,因flow()接收numpy数组为参数
                x=x.reshape((1,) + x.shape)     #要求为4维

                #使用for循环迭代,生成图片
                i = 0
                for batch in data_gen.flow(x,batch_size=1,
                                        save_to_dir=fold_new_path,
                                        save_prefix='rotation',
                                        save_format='jpeg'):
                    i += 1
                    if i>0:
                        break

            #白化与亮度
            else:
                data_gen = ImageDataGenerator(
                    rescale=1. / 255,
                    rotation_range=3,
                    width_shift_range=0.03,
                    height_shift_range=0.03,
                    shear_range=0.03,
                    zoom_range=0.03,
                    horizontal_flip=True,
                    zca_whitening=True,
                    brightness_range=[0.85, 1.15],
                    fill_mode='nearest')
                img=load_img(old_path+'/'+img_old)
                x = img_to_array(img,data_format="channels_last")   #图片转化成array类型,因flow()接收numpy数组为参数
                x=x.reshape((1,) + x.shape)     #要求为4维

                #使用for循环迭代,生成图片
                i = 0
                for batch in data_gen.flow(x,batch_size=1,
                                        save_to_dir=fold_new_path,
                                        save_prefix='white',
                                        save_format='jpeg'):
                    i += 1
                    if i>0:
                        break

In [10]:
#集成函数
old_train_path = 'D:/python code/NTU/data/Alzheimer_s Dataset/train'
new_train_path = 'D:/python code/NTU/new data/Alzheimer_s Dataset/train'
all(old_train_path,new_train_path)

  2%|▏         | 13/717 [00:00<00:05, 124.11it/s]d:\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
d:\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
d:\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
100%|██████████| 1792/1792 [00:17<00:00, 102.98it/